# QA over SQL data

## Intro
* We will create a Q&A app over tabular data in databases.
* These app will allow us to **ask a question about the data in a database in natural language and get back an answer also in natural language**.
* Building Q&A systems of SQL databases requires executing model-generated SQL queries. There are inherent risks in doing this. **In production, make sure that your database connection permissions** are always scoped as narrowly as possible for your chain's needs.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-qa-from-sql.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-qa-from-sql**

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install python-dotenv

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install langchain-community

## Connect with the database
* In this example we will use a SQLite connection with the `street_tree_db` database we have in the `data` folder.
* **Remember to download the data folder from the Github repository**.
* The `data` folder must be in the root directory of this app.
* We can interface with the database using the SQLAlchemy-driven `SQLDatabase` class:

In [6]:
from langchain_community.utilities import SQLDatabase

sqlite_db_path = "data/street_tree_db.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

We can create a simple chain that takes a question and does the following:
* Convert the question into a SQL query;
* Execute the query;
* Use the result to answer the original question.

The first step in a SQL chain is to take the user input and convert it to a SQL query. LangChain comes with a **built-in chain** for this, `create_sql_query_chain`:

#### Step 1: Translating a question in natural language into an SQL query.

In [16]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)

response = chain.invoke({"question": "How many species of trees are in San Francisco? Give the count"})

response

'```sql\nSQLQuery: SELECT COUNT(DISTINCT "qSpecies") AS "SpeciesCount" FROM street_trees;\n```'

In [27]:
def clean_sql_response(response: str) -> str:
    # Quitar el bloque de código Markdown (```sql ... ```)
    if response.startswith("```sql"):
        response = response.strip("`").split("\n", 1)[1]  # Quita primera línea (```sql)
        response = response.rsplit("\n", 1)[0]  # Quita última línea (```)

    # Quitar prefijo SQLQuery:
    if response.strip().startswith("SQLQuery:"):
        response = response.strip()[len("SQLQuery:"):].strip()

    return response

In [30]:
clean_sql_response(response)

'SELECT COUNT(DISTINCT "qSpecies") AS "SpeciesCount" FROM street_trees;'

* We can execute the query to make sure it's valid:

In [32]:
db.run(clean_sql_response(response))

'[(148,)]'

* We can also inspect the chain directly for its prompts. Pay attention to the second paragraph, where it says: **"Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database."** This is a limitation and if it is not handled well can cause errors.

In [14]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

#### Step 2: Executing the SQL query.

* Now that we've generated a SQL query, we'll want to execute it.
* When you are in production, consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution.
* We can **use the QuerySQLDatabaseTool to easily add query execution to our chain**:
    * The user asks a question.
    * The write_query chain has que question as input and the SQL query as output.
    * The execute_query chain has the SQL query as input and the SQL query execution as output.

In [39]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
write_query_1 = clean_sql_response(write_query)

execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query_1| execute_query

chain.invoke({"question": "List the species of trees that are present in San Francisco"})

AttributeError: 'RunnableSequence' object has no attribute 'startswith'

In [40]:
from langchain_core.runnables import RunnableLambda
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query_chain = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

clean_and_run = (
    write_query_chain
    | RunnableLambda(clean_sql_response)
    | RunnableLambda(execute_query.run)
)

response = clean_and_run.invoke({"question": "List the species of trees that are present in San Francisco"})
print(response)


[("Arbutus 'Marina' :: Hybrid Strawberry Tree",), ('Afrocarpus gracilior :: Fern Pine',), ("Thuja occidentalis 'Emerald' :: Emerald Arborvitae",), ("Magnolia grandiflora 'Little Gem' :: Little Gem Magnolia",), ('Platanus x hispanica :: Sycamore: London Plane',)]


#### Step 3: Translate the SQL response into a natural language response

* Now that we've got a way to generate and execute queries, we need to **combine the original question and SQL query result with the chat model to generate a final answer in natural language**.
* We can do this by passing question and result to the LLM like this:

In [106]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

sqlite_db_path = "data/street_tree_db.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

llm = ChatOpenAI(model="gpt-3.5-turbo")

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, 
    corresponding SQL query, and SQL result, 
    answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

write_query = create_sql_query_chain(llm, db)

execute_query = QuerySQLDataBaseTool(db=db)

#chain = write_query | execute_query

#chain.invoke({"question": "List the species of trees that are present in San Francisco"})

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "List the species of trees that are present in San Francisco"})

"The species of trees present in San Francisco are as follows: \n- Arbutus 'Marina' :: Hybrid Strawberry Tree\n- Afrocarpus gracilior :: Fern Pine\n- Thuja occidentalis 'Emerald' :: Emerald Arborvitae\n- Magnolia grandiflora 'Little Gem' :: Little Gem Magnolia\n- Platanus x hispanica :: Sycamore: London Plane\n- Ulmus parvifolia :: Chinese Elm\n- Leptospermum laevigatum :: Australian Tea Tree\n- Olea europaea 'Fruitless' :: Fruitless Olive\n- Ficus rubiginosa :: Port Jackson Fig\n- Cycas revoluta :: Sago palm\n- Lophostemon confertus :: Brisbane Box\n- Podocarpus gracilor :: Fern Pine\n- Eriobotrya deflexa :: Bronze Loquat\n- Ligustrum lucidum :: Glossy Privet\n- Syagrus romanzoffianum :: Queen Palm\n- Ficus microcarpa nitida 'Green Gem' :: Indian Laurel Fig Tree 'Green Gem'\n- Prunus cerasifera :: Cherry Plum\n- Myoporum laetum :: Myoporum\n- Tristanopsis laurina :: Swamp Myrtle\n- Ginkgo biloba :: Maidenhair Tree\n- Pyrus calleryana :: Ornamental Pear\n- Morus alba 'Fruitless' :: Fru

In [46]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

# 🧽 Limpia la respuesta SQL generada por el LLM
def clean_sql_response(response: str) -> str:
    if response.startswith("```sql"):
        response = response.strip("`").split("\n", 1)[1]
        response = response.rsplit("\n", 1)[0]
    if response.strip().startswith("SQLQuery:"):
        response = response.strip()[len("SQLQuery:"):].strip()
    return response

# 🧠 Junta pregunta, query y resultado en un solo diccionario
def enrich_with_result(question: str) -> dict:
    raw_query = write_query.invoke({"question": question})
    cleaned_query = clean_sql_response(raw_query)
    result = execute_query.run(cleaned_query)
    return {
        "question": question,
        "query": cleaned_query,
        "result": result
    }

# 🔧 Configuración
sqlite_db_path = "data/street_tree_db.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")
llm = ChatOpenAI(model="gpt-4o-mini")

# 🧠 Prompt para redactar la respuesta final
answer_prompt = PromptTemplate.from_template(
    """Given the following user question,
corresponding SQL query, and SQL result,
answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:"""
)

# 🔧 Subchains
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

# 🔗 Cadena final como Runnable
final_chain = (
    RunnableLambda(enrich_with_result)
    | answer_prompt
    | llm
    | StrOutputParser()
)

# 🧪 Prueba
response = final_chain.invoke("List the species of trees that are present in San Francisco. Don't limit the answer to 5")
print(response)


The species of trees that are present in San Francisco include:

- Arbutus 'Marina' :: Hybrid Strawberry Tree
- Afrocarpus gracilior :: Fern Pine
- Thuja occidentalis 'Emerald' :: Emerald Arborvitae
- Magnolia grandiflora 'Little Gem' :: Little Gem Magnolia
- Platanus x hispanica :: Sycamore: London Plane
- Ulmus parvifolia :: Chinese Elm
- Leptospermum laevigatum :: Australian Tea Tree
- Olea europaea 'Fruitless' :: Fruitless Olive
- Ficus rubiginosa :: Port Jackson Fig
- Cycas revoluta :: Sago palm
- Lophostemon confertus :: Brisbane Box
- Podocarpus gracilor :: Fern Pine
- Eriobotrya deflexa :: Bronze Loquat
- Ligustrum lucidum :: Glossy Privet
- Syagrus romanzoffianum :: Queen Palm
- Ficus microcarpa nitida 'Green Gem' :: Indian Laurel Fig Tree 'Green Gem'
- Prunus cerasifera :: Cherry Plum
- Myoporum laetum :: Myoporum
- Tristaniopsis laurina :: Swamp Myrtle
- Ginkgo biloba :: Maidenhair Tree
- Pyrus calleryana :: Ornamental Pear
- Morus alba 'Fruitless' :: Fruitless Mulberry
- Ma

In [47]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

parser = StrOutputParser()

# 🔧 Configuración
sqlite_db_path = "data/street_tree_db.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")
llm = ChatOpenAI(model="gpt-4o-mini")

# 🤖 Prompt para construir la respuesta
answer_prompt = PromptTemplate.from_template(
    """Given the following user question,
corresponding SQL query, and SQL result,
answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:"""
)

# 🧽 Función para limpiar respuesta SQL generada
def clean_sql_response(response: str) -> str:
    if response.startswith("```sql"):
        response = response.strip("`").split("\n", 1)[1]
        response = response.rsplit("\n", 1)[0]
    if response.strip().startswith("SQLQuery:"):
        response = response.strip()[len("SQLQuery:"):].strip()
    return response

# ⛓ Subchains
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

# 🔁 Función principal que ejecuta todo el flujo y retorna dict estructurado
def pipeline(question: str) -> dict:
    raw_query = write_query.invoke({"question": question})
    cleaned_query = clean_sql_response(raw_query)
    result = execute_query.run(cleaned_query)

    # Construye respuesta con LLM
    final_prompt = answer_prompt.format(
        question=question,
        query=cleaned_query,
        result=result
    )
    answer = llm.invoke(final_prompt).content

    return {
        "query": cleaned_query,
        "result": result,
        "answer": answer
    }

# 🔗 Runnable final
final_chain = RunnableLambda(pipeline)

# 🧪 Ejemplo
response = final_chain.invoke("List the species of trees that are present in San Francisco")
print(response)


{'query': 'SELECT DISTINCT "qSpecies" FROM street_trees ORDER BY "qSpecies" LIMIT 5;', 'result': '[(\'::\',), (\':: To Be Determine\',), (\':: Tree\',), ("Acacia baileyana \'Purpurea\' :: Purple-leaf Acacia",), ("Acacia baileyana :: Bailey\'s Acacia",)]', 'answer': 'The species of trees present in San Francisco include:\n\n1. \'::\' (unknown or unspecified species)\n2. \':: To Be Determine\' (species not yet determined)\n3. \':: Tree\' (generic term or unspecified type)\n4. "Acacia baileyana \'Purpurea\' :: Purple-leaf Acacia"\n5. "Acacia baileyana :: Bailey\'s Acacia" \n\nPlease note that some entries may appear to be placeholder names or unspecified species.'}


In [104]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

parser = StrOutputParser()

# 🔧 Configuración
sqlite_db_path = "data/street_tree_db.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")
llm = ChatOpenAI(model="gpt-4o-mini")

# 🤖 Prompt para construir la respuesta
answer_prompt = PromptTemplate.from_template(
    """Given the following user question,
corresponding SQL query, and SQL result,
answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:"""
)

# 🧽 Función para limpiar respuesta SQL generada
def clean_sql_response(response: str) -> str:
    response = response.strip()

    # Eliminar bloque de código markdown ```sql ... ```
    if response.startswith("```sql"):
        lines = response.splitlines()
        lines = [line for line in lines if not line.strip().startswith("```")]
        response = "\n".join(lines)

    # Eliminar prefijo 'SQLQuery:' si lo incluye
    if response.startswith("SQLQuery:"):
        response = response[len("SQLQuery:"):].strip()

    return response.strip()

# ⛓ Subchains
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

# 🔁 Función principal que ejecuta todo el flujo y retorna dict estructurado
def pipeline(question: str) -> dict:
    raw_query = write_query.invoke({"question": question})
    cleaned_query = clean_sql_response(raw_query)
    try:
        result = execute_query.run(cleaned_query)
    except Exception as e:
        result = f"[SQL ERROR] {str(e)}"


    # Construye respuesta con LLM
    final_prompt = answer_prompt.format(
        question=question,
        query=cleaned_query,
        result=result
    )
    #llm_response = llm.invoke(final_prompt)
    #answer = parser.invoke(llm_response)
    answer = (llm | parser).invoke(final_prompt)


    return {
        "query": cleaned_query,
        "result": result,
        "answer": answer
    }

# 🔗 Runnable final
final_chain = RunnableLambda(pipeline)

# 🧪 Ejemplo
response = final_chain.invoke("List the species of trees that are present in San Francisco with the number of trees for each species group by the name of the species. show all the results")
#print(response)


In [105]:
print(response["answer"])

The species of trees present in San Francisco and the corresponding number of trees for each species are as follows:

1. `::` - 5 trees
2. `:: To Be Determine` - 4 trees
3. `:: Tree` - 10 trees
4. `Acacia baileyana 'Purpurea' :: Purple-leaf Acacia` - 1 tree
5. `Acacia baileyana :: Bailey's Acacia` - 1 tree
6. `Acacia decurrens :: Acacia: Silver Wattle` - 1 tree
7. `Acacia melanoxylon :: Blackwood Acacia` - 18 trees
8. `Acacia spp :: Acacia Spp` - 1 tree
9. `Acacia stenophylla :: Shoestring Acacia` - 11 trees
10. `Acer buergeranum :: Trident Maple` - 4 trees
11. `Acer japonicum :: Japanese Maple` - 1 tree
12. `Acer palmatum :: Japanese Maple` - 5 trees
13. `Acer rubrum 'Armstrong' :: Armstrong Red Maple` - 1 tree
14. `Acer rubrum 'October Glory' :: Red October Glory Maple` - 5 trees
15. `Acer rubrum :: Red Maple` - 11 trees
16. `Aesculus x carnea 'Briotii' :: Ruby Horse Chestnut` - 1 tree
17. `Aesculus x carnea :: Red Horse Chestnut` - 5 trees
18. `Afrocarpus gracilior :: Fern Pine` - 4

In [80]:
print(response["query"])

```sql
SELECT "qSpecies", COUNT(*) AS "TreeCount" 
FROM street_trees 
GROUP BY "qSpecies";
```


In [86]:
db.run("SELECT qSpecies, COUNT(*) AS TreeCount FROM street_trees GROUP BY qSpecies;")

'[(\'::\', 5), (\':: To Be Determine\', 4), (\':: Tree\', 10), ("Acacia baileyana \'Purpurea\' :: Purple-leaf Acacia", 1), ("Acacia baileyana :: Bailey\'s Acacia", 1), (\'Acacia decurrens :: Acacia: Silver Wattle\', 1), (\'Acacia melanoxylon :: Blackwood Acacia\', 18), (\'Acacia spp :: Acacia Spp\', 1), (\'Acacia stenophylla :: Shoestring Acacia\', 11), (\'Acer buergeranum :: Trident Maple\', 4), (\'Acer japonicum :: Japanese Maple\', 1), (\'Acer palmatum :: Japanese Maple\', 5), ("Acer rubrum \'Armstrong\' :: Armstrong Red Maple", 1), ("Acer rubrum \'October Glory\' :: Red October Glory Maple", 5), (\'Acer rubrum :: Red Maple\', 11), ("Aesculus x carnea \'Briotii\' :: Ruby Horse Chestnut", 1), (\'Aesculus x carnea :: Red Horse Chestnut\', 5), (\'Afrocarpus gracilior :: Fern Pine\', 4), (\'Agonis flexuosa :: Peppermint Willow\', 3), (\'Araucaria heterophylla :: Norfolk Island Pine\', 1), ("Arbutus \'Marina\' :: Hybrid Strawberry Tree", 39), (\'Arbutus unedo :: Strawberry Tree\', 2), (\

In [129]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

# 📦 Conexión a base de datos
db = SQLDatabase.from_uri("sqlite:///data/street_tree_db.sqlite")
llm = ChatOpenAI(model="gpt-4o-mini")

# 🧽 Limpieza robusta de la respuesta SQL generada# Limpieza con impresión de depuración
def clean_sql_response(response: str) -> str:
    import re

    # Eliminar bloque Markdown ```sql ... ``` incluso si hay espacios o saltos
    response = response.strip()

    # Usar regex para eliminar bloques como ```sql ... ```
    response = re.sub(r"```sql\s*([\s\S]+?)\s*```", r"\1", response, flags=re.IGNORECASE).strip()

    # Eliminar prefijo SQLQuery:
    if response.startswith("SQLQuery:"):
        response = response[len("SQLQuery:"):].strip()

    return response



# 🔐 Ejecución segura del SQL con captura de errores
def safe_execute_query(query: str) -> str:
    try:
        return execute_query.run(query)
    except Exception as e:
        return f"[SQL ERROR] {str(e)}"

# ✂️ Limpieza de Markdown del output del modelo (en caso de que ignore el prompt)
def clean_markdown_blocks(text: str) -> str:
    return text.replace("```sql", "").replace("```", "").strip()

# 🧠 Prompt ajustado para evitar SQL y markdown en la respuesta
answer_prompt = PromptTemplate.from_template(
    """You are a helpful data assistant.

Given the following user question,
corresponding SQL query, and SQL result,
answer the user question.

- DO NOT include SQL markdown formatting (no ```sql or ```).
- DO NOT repeat the SQL query in your answer.
- If the result starts with [SQL ERROR], explain briefly what likely went wrong and optionally suggest a fix.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:"""
)

# 🧱 Componentes
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

# Parser con limpieza adicional post-LLM
parser = RunnableLambda(lambda x: clean_markdown_blocks(StrOutputParser().invoke(x)))

# 🔗 Cadena completa
chain = (
    RunnablePassthrough.assign(
        query=write_query | RunnableLambda(clean_sql_response)
    ).assign(
        result=itemgetter("query") | RunnableLambda(safe_execute_query)
    )
    | answer_prompt
    | llm
    | parser
)

# 🧪 Ejemplo de uso
response = chain.invoke({
    "question": "List the species of trees that are present in San Francisco, and the number of trees per species. Show all the results not just 5."
})

print(response)



The species of trees present in San Francisco and their respective counts are as follows:

- '::' - 5 trees
- ':: To Be Determine' - 4 trees
- ':: Tree' - 10 trees
- "Acacia baileyana 'Purpurea' :: Purple-leaf Acacia" - 1 tree
- "Acacia baileyana :: Bailey's Acacia" - 1 tree
- 'Acacia decurrens :: Acacia: Silver Wattle' - 1 tree
- 'Acacia melanoxylon :: Blackwood Acacia' - 18 trees
- 'Acacia spp :: Acacia Spp' - 1 tree
- 'Acacia stenophylla :: Shoestring Acacia' - 11 trees
- 'Acer buergeranum :: Trident Maple' - 4 trees
- 'Acer japonicum :: Japanese Maple' - 1 tree
- 'Acer palmatum :: Japanese Maple' - 5 trees
- "Acer rubrum 'Armstrong' :: Armstrong Red Maple" - 1 tree
- "Acer rubrum 'October Glory' :: Red October Glory Maple" - 5 trees
- 'Acer rubrum :: Red Maple' - 11 trees
- "Aesculus x carnea 'Briotii' :: Ruby Horse Chestnut" - 1 tree
- 'Aesculus x carnea :: Red Horse Chestnut' - 5 trees
- 'Afrocarpus gracilior :: Fern Pine' - 4 trees
- 'Agonis flexuosa :: Peppermint Willow' - 3 tr

#### Let's review what is happening in the above chain.
* The user asks a question (identified by the variable name "question").
* We use RunnablePassthrough to get that "question" variable, and we use .assign() twice to get the other two variables required by the prompt template: "query" and "result".
* With the first .assign(), the write_query chain has que question as input and the SQL query (identified by the variable name "query") as output.
* With the second .assign(), the execute_query chain has the SQL query as input and the SQL query execution (identified by the variable name "result") as output.
* The prompt template has the question (identified by the variable name "question"), the SQL query (identified by the variable name "query") and the SQL query execution (identified by the variable name "result") as input, and the final prompt as the output.
* The chat model has the prompt as he input and the AIMessage with the response in natural language as the output.
* The StrOutputParser has the AIMessage with the response in natural language as the input and the response in natural language as a string of text as the output.

#### About the role of .assign() in this chain
In this exercise we have learned more about the .assign() built-in method for Runnables. We have seen that the .assign() method **allows us to include additional keys and values** based on existing input.
* With the first .assign(), the write_query chain has the question as input and the SQL query (identified by the variable name "query") as output.
* With the second .assign(), the execute_query chain has the SQL query as input and the SQL query execution (identified by the variable name "result") as output.

The .assign() method allows us to add new keys and values to the output of a Runnable **while keeping the original keys and values intact**. See the process again:
* We use RunnablePassthrough to get that "question" variable, and we use .assign() twice to get the other two variables required by the prompt template: "query" and "result".
* With the first .assign(), the write_query chain has que question as input and the SQL query (identified by the variable name "query") as output.
* With the second .assign(), the execute_query chain has the SQL query as input and the SQL query execution (identified by the variable name "result") as output.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 004-invoke-stream-batch.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-qa-from-sql.py